In [ ]:
!pip install PyPDF2 #Extracts text from PDFs with selectable text layers.
!pip install pymupdf # rendering pages as images and extracting embedded images
!pip install pytesseract
!apt-get install tesseract-ocr
!pip install pycryptodome
!pip install easyocr

# !apt-get install tesseract-ocr-[language_code]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import pandas as pd
import easyocr
from Crypto.Cipher import AES
import os  # File and folder operations
from PyPDF2 import PdfReader  # Text layer extraction
import fitz  # PyMuPDF for handling PDF content (text and images)
from PIL import Image  # Image handling and saving
from pytesseract import image_to_string  # OCR text extraction from images
import io  # Handle in-memory image data
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# files in shared drive
# !ls "/content/drive/My Drive"
# pdf_folder="/content/drive/My Drive/polsky doc/"
pdf_folder="/content/drive/My Drive/polsky file split/folder_3"
# pdf_folder="/content/drive/My Drive/test/"
if os.path.exists(pdf_folder):
  print("total file count")
  print(len(os.listdir(pdf_folder)))

total file count
1067


In [ ]:
# text extract function
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Use PyPDF2 for text extraction
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text()
    except Exception as e:
        print(f"PyPDF2 failed for {pdf_path}: {e}")

    return text

# scan contents: ignore text layer that might exist
def extract_scanned_pages(pdf_path):
    ocr_text = ""
    try:
        doc = fitz.open(pdf_path) # parse entire file by each page
        for page_number in range(len(doc)):
            pix = doc[page_number].get_pixmap()  # Render entire page as an image
            img = Image.open(io.BytesIO(pix.tobytes()))  # Convert to PIL image

            # Perform OCR on the image
            page_ocr_text = image_to_string(img)
            ocr_text += f"--- Page {page_number + 1} OCR ---\n{page_ocr_text}\n"
    except Exception as e:
        print(f"OCR failed for {pdf_path}: {e}")
    return ocr_text

# def extract_scanned_pages(pdf_path, lang_list=['en'], use_gpu=True): # easyocr method
#     ocr_text = ""
#     try:
#         # Initialize EasyOCR Reader
#         reader = easyocr.Reader(lang_list, gpu=use_gpu)

#         doc = fitz.open(pdf_path)

#         for page_number in range(len(doc)):
#             # Render the page as an image
#             pix = doc[page_number].get_pixmap()
#             img = Image.open(io.BytesIO(pix.tobytes()))  # Convert to PIL image

#             page_ocr_text = reader.readtext(img, detail=0)  # Get OCR text, 'detail=0' gives plain text

#             ocr_text += f"--- Page {page_number + 1} OCR ---\n{' '.join(page_ocr_text)}\n"

#     except Exception as e:
#         print(f"OCR failed for {pdf_path}: {e}")

#     return ocr_text




In [ ]:
# pasring
def parse_pdf_with_embedded_images(pdf_path):

    # print(f"Processing {pdf_path}...")

    # Get the base filename
    base_filename = os.path.splitext(os.path.basename(pdf_path))[0]

    # Extract text layer
    text = extract_text_from_pdf(pdf_path)

    # Extract OCR from scanned pages, only executed if length of text part is less than 100 words
    # Initialize OCR text
    ocr_text = ""
    if not text.strip() or len(text.split()) <=100:
        ocr_text = extract_scanned_pages(pdf_path)

    return base_filename, text, ocr_text




In [ ]:
export_folder="/content/drive/My Drive/polsky file split/"

results = {}

# Check if the PDF folder exists
if not os.path.exists(pdf_folder):
    print("PDF folder not found!")
else:
    # Iterate through files in the folder
    # for filename in sorted(os.listdir(pdf_folder)) # use sort
    count=0
    for filename in os.listdir(pdf_folder):
        file_path = os.path.join(pdf_folder, filename)

        # Process only PDF files
        if filename.lower().endswith(".pdf"):
            # Call the function and store the results in the dictionary
            base_filename, text, ocr_text = parse_pdf_with_embedded_images(file_path)
            results[base_filename] = {
                "text": text,
                "ocr_text": ocr_text,
            }
        count+=1
        print(count)
# Print the results dictionary (optional, for verification)
print("Processing completed!")


1
2


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame.from_dict(results, orient='index').reset_index()
output_path=os.path.join(export_folder,"export_3.csv")
df.to_csv(output_path, index=False, escapechar='\\')
df

In [ ]:
len(df)


# folder split

In [ ]:

# import os
# import shutil

# # Source folder containing the 20,000 files
# source_folder = "/content/drive/My Drive/polsky doc"

# # Destination parent folder for subfolders
# destination_folder = "/content/drive/My Drive/polsky file split"

# # Number of subfolders to create
# num_folders = 20

# # Check if source folder exists
# if not os.path.exists(source_folder):
#     print("Source folder not found!")
# else:
#     # Get all files in the source folder
#     all_files = sorted(os.listdir(source_folder))  # Sort for consistent distribution

#     # Calculate the number of files per folder
#     files_per_folder = len(all_files) // num_folders
#     remaining_files = len(all_files) % num_folders  # Handle uneven distribution

#     # Create subfolders and distribute files
#     for folder_index in range(num_folders):
#         # Create a subfolder
#         subfolder_name = f"folder_{folder_index + 1}"
#         subfolder_path = os.path.join(destination_folder, subfolder_name)
#         os.makedirs(subfolder_path, exist_ok=True)

#         # Calculate file indices for this subfolder
#         start_idx = folder_index * files_per_folder
#         end_idx = start_idx + files_per_folder

#         # Add one extra file to folders if there are remaining files
#         if folder_index < remaining_files:
#             end_idx += 1

#         # Move files to the subfolder
#         for file in all_files[start_idx:end_idx]:
#             src_file_path = os.path.join(source_folder, file)
#             dest_file_path = os.path.join(subfolder_path, file)
#             shutil.move(src_file_path, dest_file_path)

#         print(f"Moved files to {subfolder_path}")

#     print("Files successfully distributed into subfolders!")
